pytorch公式の畳み込みニューラルネットワークのexampleコードにコメントを追加したものです．

サンプルの畳み込みニューラルネットワークは，畳み込み層2つ，全結合層2つで構成されています．最初の畳み込み層conv1の出力はReLu関数を通し，次の畳み込み層conv2に入力されます．conv2の出力はReLu関数と通した後，Max poolingを行います．poolingと全結合層fc1の間は学習時dropout処理がされます．最初の全結合層fc1の出力はReLu関数を通し，次の全結合層fc2に入力されます．fc1とfc2間で学習時dropoutが起こります．fc2の出力はsoftmax関数により規格化されます．

dropout: 学習時，任意の割合の入力ユニットの出力を0にします．それにより過学習を回避することが期待できます．しかし効果があるとは限りません．

In [10]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # まず最初の畳み込み層には1チャネルの入力が入る．画像認識の場合，入力のチャネルとは色数と思っておく．
        # チャンネルが1つのときはグレースケール（輝度情報しかない）画像が入力で，チャネルが3つのときはRGBカラー画像が入力である．
        # 32は出力のチャネル数である．言い方を変えればカーネルの種類の数である．32種類のカーネルを適用した画像がこの畳込み層で出力されるといえる．詳しくは他のウェブサイトを参考にしてほしい．
        # 3は畳み込みのサイズ（カーネルサイズ）で，1はカーネルをズラす量である．詳しくは他のウェブサイトを参考にしてほしい．
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25) # dropout1では0.25の確率で出力の値が0になる．
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128) # fc1は入力9216，出力128の中間層が作成される．
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x) # conv1の出力はReLu関数と通しconv2に入力される．
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2) # ReLu関数を通したconv2の出力はmax poolongされる．
        x = self.dropout1(x)
        x = torch.flatten(x, 1) # 出力が1次元化される．
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        # fc2の出力はsoftmax関数にかけられる．しかし，誤差にcross entropyを用いる場合，誤差計算でlogの計算をするのではなく出力の段階でlogを計算する．
        output = F.log_softmax(x, dim=1)
        return output

In [12]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    # ミニバッチで学習がされる．1度の学習ですべてのデータを使うのではなく一部のデータを使う．
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # 面白いことにtargetはラベル（例えば，1，2，...）なのだが，pytorchは自動でone-of-k codingにしてくれる．
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target) # 誤差を計算している．誤差関数はcross entropyだが，先にlogの計算を行っているのでnll_lossが用いられている．
        loss.backward()
        optimizer.step()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [13]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                                                       # logは単調増加関数なので，softmaxの最大値を取るユニットはlog softmaxの最大値を取るユニットと等しい．
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [14]:


train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}

# 入力画像の編集を行う関数．この例では規格化しかしていないが，回転などの処理も追加できる．
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

dataset1 = datasets.MNIST('.', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('.', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)


In [15]:
device = "cpu" # 計算するデバイスを指定している．gpuの方が計算が早く終る．
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.5) # この例では最適化法としてAdadeltaが使われている．

Epochs = 10 # いわゆる，学習回数．
scheduler = StepLR(optimizer, step_size=1, gamma=0.7) # StepLRを用いることで学習回数が増えると学習率が減るようになる．


In [ ]:
for epoch in range(1, Epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()